Calculs de la moyenne et de l'écart type par bande sur une série d'images

In [1]:
from utils.utils import (get_environment,get_root_path,update_storage_access)
import os
import s3fs
from classes.data.satellite_image import SatelliteImage
import numpy as np

In [11]:
update_storage_access()
root_path = get_root_path()
environment = get_environment()

src = "SENTINEL2"
date = 2022
dep = "976"
nb_bands = 12

In [ ]:
bucket = environment["bucket"]
path_s3 = environment["sources"][src][date][dep]
path_local = os.path.join(
    root_path, environment["local-path"][src][date][dep]
)

fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}
)
fs.download(
    rpath=f"{bucket}/{path_s3}", lpath=f"{path_local}", recursive=True
)

In [ ]:
path_local = ''
list_names = os.listdir(path_local)
list_paths = [path_local + '/' + name for name in list_names]

nb_images = len(list_paths)
mean = [0]*nb_bands
std = [0]*nb_bands

for path in list_paths:
    si = SatelliteImage.from_raster(
        file_path=path,
        dep=dep,
        date=date,
        n_bands=nb_bands
    )

    for i in range(nb_bands):
        if not np.isnan(np.mean(si.array[i])):
            mean[i] += np.mean(si.array[i])
            std[i] += np.std(si.array[i])

for i in range(nb_bands):
    mean[i] /= nb_images
    std[i] /= nb_images

mean=tuple(mean)
std=tuple(std)

print(mean, std)

Calcul et export en .tif du label correspondant à une image d'entrée

In [ ]:
from datetime import datetime
import numpy as np
from tqdm import tqdm
import os

from classes.data.satellite_image import SatelliteImage

from train_pipeline_utils.prepare_data import (
    label_images,
    save_images_and_masks,
)
from classes.labelers.labeler import BDTOPOLabeler
from utils.utils import remove_dot_file

In [ ]:
year = 2022
dep = "971"
nb_bands=12
task="segmentation"

In [ ]:
list_output_dir = []
output_dir = "img4b"
date = datetime.strptime(str(year) + "0101", "%Y%m%d")

dir = '/home/onyxia/work/detection-habitat-spontane/donnees-test'
list_path = [dir + "/" + filename for filename in remove_dot_file(os.listdir(dir))]
liste_sorted = np.sort(list_path)
labeler = BDTOPOLabeler(date, dep)

for path in tqdm(liste_sorted):
    si = SatelliteImage.from_raster(
        file_path=path,
        dep=dep,
        date=date,
        n_bands=nb_bands,
    )
    filename = path.split("/")[-1].split(".")[0]

    labels, balancing_dict = label_images(
        [si], labeler, task=task
    )

    save_images_and_masks(
        [si],
        labels,
        '/home/onyxia/work/detection-habitat-spontane/src/img4b',
        '/home/onyxia/work/detection-habitat-spontane/donnees-test',
        task=task,
    )